In [3]:
import pandas as pd
import requests
import bs4 #beautifulsoup4
import re
import datetime

In [9]:
url = 'https://www.coursereport.com/best-coding-bootcamps'
response = requests.get(url)

In [12]:
html_bytes = response.content

In [16]:
soup = bs4.BeautifulSoup(html_bytes, "html.parser")

In [17]:
type(soup)

bs4.BeautifulSoup

In [18]:
bs4.BeautifulSoup

bs4.BeautifulSoup

In [19]:
soup.body

<body data-spy="scroll" data-target="#toc" style="position:relative;"><header><nav class="navbar navbar-default" role="navigation"><div class="container" id="nav-container" itemscope="" itemtype="http://schema.org/Organization"><div class="navbar-header"><button class="navbar-toggle collapsed" data-target="#navbar-collapse" data-toggle="collapse" type="button"><span class="sr-only">Toggle navigation</span><span class="icon-bar"></span><span class="icon-bar"></span><span class="icon-bar"></span></button><a class="navbar-brand logo-small" href="https://www.coursereport.com" itemprop="url"><img alt="Course Report" itemprop="logo" src="https://coursereport-production-herokuapp-com.global.ssl.fastly.net/assets/logo-small-756b1c93378e1fdcc7f90674634f7675.png" title="Course Report"/></a></div><div class="collapse navbar-collapse" id="navbar-collapse"><ul class="nav navbar-nav"><li class="top-level-nav" id="schools"><a class="mp-gb-header" href="/schools">Browse Schools</a><ul class="tracks"><

In [37]:
# We want to get the following columns from coursereport.com to create a table:
# rank, name of school, rating, stars, no of reviews, cities
# Let's start with get the rank and name
# find method takes a string and tag name:

(soup
.body
.find("div", class_='main-body')
.find("div", class_='longform-body container')
.find("div", class_='row')
.find("div", class_='col-md-11')
.find("div", class_='longform-body container')
.find("ul", id ='schools')
.find_all('li')
)

AttributeError: 'NoneType' object has no attribute 'find'

In [108]:
school_list_items = soup.find("ul", id ='schools')

In [111]:
# Let's see if we could have skipped all the previous entries and go straight to the url tag with id 'schools'

school_list_items = (soup
                      .find('ul', id='schools')
                      .find_all('li'))

In [112]:
flatiron = school_list_items[0]

In [113]:
# Get the rank and name


flatiron.h3.a.contents[0]
rank_name=flatiron.h3.a.contents[0]
rank_name

'1. Flatiron School'

In [93]:
# Extrakt rank and name of school individually using regex:

rank_pattern = r"^\d{1,2}"
rank = re.findall(rank_pattern, rank_name)[0]
print(rank)

#name
# \d{1,2} = digit 1 or 2
# . = regex for any character
# + = for anything else

name_pattern = r"^\d{1,2}\.\s(.+)$"
name = re.findall(name_pattern, rank_name)[0]
print(name)

1
Flatiron School


In [94]:
# Get the rating:

rating_string = flatiron.find('span', class_='longform-rating-text').contents[0]
rating_string

'Overall Rating: (4.75) '

In [95]:
# Extrakt rating:

rating_pattern = r"\((.+)\)"
rating = float(re.findall(rating_pattern, rating_string)[0])
rating

4.75

In [96]:
# Get the stars:

(flatiron
 .find('div', class_='ratings title-rating')
 .find_all('span'))[1:][0]['class']

['icon-full_star']

In [97]:
# extrakt stars:

# 1. get a list of all stars

stars_list = [tag['class'][0] for tag in (flatiron
                                      .find('div', class_='ratings title-rating')
                                      .find_all('span'))[1:]]

stars_list

# 2. translate text into numeric values
# 2.1 create a dict with translations:

star_dict = {'icon-full_star': 1,
            'icon-half_star': 0.5,
            'icon-empty_star': 0}

# 2.2 translate stars to floats and sum list:

stars = float(sum([star_dict[s] for s in stars_list]))
stars


4.5

In [98]:
# Get the no of reviews:

(flatiron
 .find_all('div', class_='ratings title-rating')[1]
 .a
 .contents[0])

'409 Reviews'

In [99]:
reviews_string = (flatiron
                 .find_all('div', class_='ratings title-rating')[1]
                 .a
                 .contents[0])

reviews_string

'409 Reviews'

In [100]:
# extract no of reviews:

reviews_pattern = r"^\d+"
reviews = int(re.findall(reviews_pattern, reviews_string)[0])
reviews

409

In [101]:
# Get cities:

(flatiron
 .find('span', class_='location')
 .find_all('a'))[0].contents[0]

'Online'

In [102]:
# How do we want to represent multiple cities. Let's go for a string with a unique delimiter, such as "|"

ex = "Online|New York City|Seattle"

In [103]:
ex.split("|")

['Online', 'New York City', 'Seattle']

In [104]:
cities_list = (flatiron
               .find('span', class_='location')
               .find_all('a'))
[tag.contents[0] for tag in cities_list]

cities_list = [tag.contents[0] for tag in (flatiron
                                           .find('span', class_='location')
                                           .find_all('a'))]

cities = "|".join(cities_list)

cities
                                      

'Online|New York City|Washington|Houston|Seattle|Denver|Chicago|Atlanta|London|Austin|San Francisco|Sydney'

In [105]:
# put all values in a dict:

rank_items = {"rank" : rank,
            "name" : name,
           "rating" : rating,
           "stars" : stars,
            "reviews" : reviews,
           "cities" : cities,
           }

rank_items

{'rank': '1',
 'name': 'Flatiron School',
 'rating': 4.75,
 'stars': 4.5,
 'reviews': 409,
 'cities': 'Online|New York City|Washington|Houston|Seattle|Denver|Chicago|Atlanta|London|Austin|San Francisco|Sydney'}

# Let's create some functions so we can iterate over all schools

We create functions that extract the required informations

In [114]:
hack_reactor = school_list_items[1]

In [85]:
#rank_name=flatiron.h3.a.contents[0]
#rank_name

#rank_pattern = r"^\d{1,2}"
#rank = int(re.findall(rank_pattern, rank_name)[0])
#print(rank)

#name_pattern = r"^\d{1,2}\.\s(.+)$"
#name = re.findall(name_pattern, rank_name)[0]
#print(name)



#stich previously approach together into functions

# function to get rank:
def get_rank(list_item):
    rank_name=list_item.h3.a.contents[0]
    rank_pattern = r"^\d{1,2}"
    return int(re.findall(rank_pattern, rank_name)[0])

# function to get name:
def get_name(list_item):
    rank_name=list_item.h3.a.contents[0]
    name_pattern = r"^\d{1,2}\.\s(.+)$"
    return re.findall(name_pattern, rank_name)[0]

In [119]:
print(get_rank(hack_reactor))
print(get_name(hack_reactor))

2
Hack Reactor


In [ ]:
# function to get rating:

# function to get stars:
def get_stars(list_item):
    rank_name=list_item.h3.a.contents[0]
# function to get no of reviews:

# function to get cities:

    